In [2]:
import numpy as np
import pandas as pd

In [ ]:
def answer(songs_know, lsh):
    # ------------ input -------------
    # songs_know : playlist에 있는 songs들을 담은 list
    
    
    candidate_songs = dict()
    
    for song in songs_know:
        
        #함수 이름 앵버가 짜면 바꿔야 함
        toadd = lsh.sim_items_dic(song)
        
        # candidate_songs에서 얼만큼 언급되었는지 세어준다
        for it in toadd:
            if(it in candidate_songs):
                candidate_songs[it] += 1
            else: #처음 등장한 녀석
                candidate_songs[it] = 1
    
    candidate_songs = dict(sorted(candidate_songs.items(), reverse=True, key=lambda item: item[1]))

    if(len(candidate_songs) < 100):
        print("candidate les than 100")
        real_candidate = list(candidate_songs.keys())
    else:
        real_candidate = list(candidate_songs.keys())[0:99]
    
    #for debugging 대충 몇 개정도씩 들어있는지?
    #print(sum(list(candidate_songs.values()))/ len(list(candidate_songs.values())))
    
    return real_candidate

In [ ]:
# Numpy Matrix
# Column : Each song
# Row : Shingles



# Instance variables :
# n : the number of total songs
# s : the number of signatures
# r : the number of signatures in a single band
# b : the number of bands (s // r)
# perm : permutated np array
# mhdata : minhashed (signature array) 
#          row : signatures / columns : songs
# ---------------------------------------------------------------
class LSH() :
    import numpy as np 
    # Initializer
    # Input
    # data : pandas dataframe for song - shingles
    #        row : songs / column : array of tags for each song
    # sig_n : the number of signatures
    # band_r : the number of signatures in a single band
    # bucket_sz : bucket size for LSH
    # -----------------------------------------------------------
    def __init__(self, data, sig_n, band_r) :
        self.data = data
        self.n = data.shape[0]
        self.s = sig_n
        self.r = band_r
        self.bucket_sz = min(500, 1 << self.r)
        if self.s%self.r != 0 :
            print("The number of signatures is not divided by the number of band_r.")
            print("Warning : Remainder signatures will be ignored")
            self.s = self.s - self.s%self.r
        self.b  = self.s // self.r
        self.song_dic = dict(zip(self.data.iloc[:,0],range(len(self.data.iloc[:,0]))))   #index might change

        
    # permutation
    # save random shuffle of 1:n 
    def permute(self, tag_n) :
        self.perm = np.random.permutation(tag_n)
        
    # This function uses permutation to hash, outputs the minimum value
    # tag_n : Total number of tags (required for permutation)
    # This algorithm could be improved using
    # https://datalab.snu.ac.kr/~ukang/courses/19S-DM/L5-lsh.pdf page 42 later
    def minhash(self, tag_n = None) :
        # result matrix
        # row : signatures / columns : songs
        sig_mat = np.zeros((self.s, self.n))
        for sig_num in range(self.s) :
            self.permute(tag_n)
            new_sig = [min(np.take(self.perm,i)) for i in self.data.iloc[:,1]]  # index 1 could be changed later
            sig_mat[sig_num, :] = new_sig
        self.mhdata = sig_mat
    
    # This function locally hashes the signature vector (of size self.r)
    def local_hash(self, sig_vec) :
        return hash(str(sig_vec)) % self.bucket_sz
    
    # Computes locally sensitive hashing algorithm (put into buckets)
    # Bucket_mtx : row - bands, column - bucket for each band 
    def lsh(self) :
        mh_t = np.transpose(self.mhdata)
        bucket_mtx = np.zeros((self.b, self.bucket_sz, 1))
        for band_i in self.b :
            curr = band_i * self.r 
            hashed = [self.local_hash(col[curr:curr+self.r]) for col in mh_t]
            for ind, item in enumerate(hashed, start=0): :
                bucket_mtx[band_i][item] = np.append(bucket_mtx[band_i][item], ind)
        
        self.bucket_mtx = bucket_mtx
    
    # Outputs the array of similar songs w.r.t given song
    def sim_items(self, song) :
        sim_item = np.array([])
        song_sig = self.mhdata[:,self.song_dic[song]]
        for band_i in self.b :
            curr = band_i * self.r
            curr_hash_val = self.local_hash(song_sig[curr:curr+self.r])
            bucket_for_this = self.bucket_mtx[band_i][curr_hash_val]
            sim_item = numpy.union1d(sim_item, bucket_for_this)
        return np.take(self.data.iloc[:, 0], sim_item)

In [1]:
lst = ['A','B','C']
dict(zip(lst,range(len(lst))))

{'A': 0, 'B': 1, 'C': 2}